In [ ]:
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [ ]:

# Configura Selenium para usar Chrome
service = Service(ChromeDriverManager().install())
# Dont open the browser
options = webdriver.ChromeOptions()
options.add_argument("--headless") 
# Driver
driver = webdriver.Chrome(service=service,options=options)


user = 'ComputingVictor'
# Abre la URL objetivo
driver.get(f'https://www.kaggle.com/{user}')

# Usa una espera implícita si es necesario para asegurar que la página haya cargado
driver.implicitly_wait(10)

# Encuentra el elemento por su atributo data-testid
elemento = driver.find_element(By.XPATH, '/html/body/main/div[1]/div/div[5]/div[2]/div[2]/div/div/div[5]/div[1]/div[1]')

# Utilizar expresiones regulares para extraer la información relevante
patron_medallas = re.compile(r"(Competitions|Datasets|Notebooks|Discussions)\n([A-Za-z]+)\nMEDALS\n(?:none yet|(\d+)(?:\n(\d+)(?:\n(\d+))?)?)")

# Extraer todas las coincidencias
matches = patron_medallas.findall(elemento.text)

# Convertir los matches en un diccionario adecuado para la creación de un DataFrame
datos_nuevos = {match[0]: {"Level": match[1], "Gold": 0, "Silver": 0, "Bronze": 0} for match in matches}

# Cierra el navegador
driver.quit()

# Actualizar el diccionario con la información de las medallas, ajustando según la cantidad de números encontrados
for match in matches:
    category = match[0]
    # Convertir valores de medallas a entero, usando 0 para valores faltantes
    medallas = [int(x) if x.isdigit() else 0 for x in match[2:]]
    print(medallas)
    if len(medallas) == 3:  # Oro, Plata, Bronce
        datos_nuevos[category]["Gold"], datos_nuevos[category]["Silver"], datos_nuevos[category]["Bronze"] = medallas
    elif len(medallas) == 2:  # Plata, Bronce
        datos_nuevos[category]["Silver"], datos_nuevos[category]["Bronze"] = medallas
    elif len(medallas) == 1:  # Bronce
        datos_nuevos[category]["Bronze"] = medallas[0]

# Reajustar la lógica para cumplir con el requisito exacto del usuario
for category, details in datos_nuevos.items():
    gold, silver, bronze = details["Gold"], details["Silver"], details["Bronze"]
    
    # Si bronze es 0 y hay medallas en oro o plata, deslizar las medallas hacia la derecha
    if bronze == 0 and silver == 0 :
        # Deslizar las medallas a la derecha correctamente
        details["Bronze"], details["Silver"], details["Gold"] = gold, 0, 0
    elif bronze == 0 and silver > 0:
        # Deslizar las medallas a la derecha correctamente
        details["Bronze"], details["Silver"], details["Gold"] = silver, gold, 0

# Convertir el diccionario ajustado a DataFrame para mostrar los resultados
df_ajustado_final = pd.DataFrame(datos_nuevos).T

df_ajustado_final


In [10]:
from PIL import Image, ImageDraw, ImageFont

# Cargar la imagen base llamada "coco"
img = Image.open('../img/Kaggle_template.png')

# Preparar para añadir texto
d = ImageDraw.Draw(img)
fnt = ImageFont.truetype('PPTelegraf-Regular.otf')

# La información a añadir
user = 'ComputingVictor'

# Definir la jerarquía de los niveles
jerarquia_niveles = ['Novice', 'Contributor', 'Expert', 'Master', 'Grandmaster']
nivel_usuario = df_ajustado_final['Level'].apply(lambda x: jerarquia_niveles.index(x)).idxmax()
nivel_usuario = df_ajustado_final.loc[nivel_usuario, 'Level']

total_oro = df_ajustado_final.Gold.sum()  # Suma total de medallas de oro
total_plata = df_ajustado_final.Silver.sum()  # Suma total de medallas de plata
total_bronce = df_ajustado_final.Bronze.sum()  # Suma total de medallas de bronce

# Añadir texto a la imagen
fnt = ImageFont.truetype('PPTelegraf-Regular.otf',size=40)
d.text((120,55), f"{user}", font=fnt, fill=(0,0,0),align='center',stroke_width=2)

fnt = ImageFont.truetype('PPTelegraf-Regular.otf',size=40)
d.text((220,105), f"{nivel_usuario}", font=fnt, fill=(0,0,0),align='center')

fnt = ImageFont.truetype('PPTelegraf-Regular.otf',size=40)
d.text((75,640), f"{total_oro}", font=fnt, fill=(0,0,0),align='center')

fnt = ImageFont.truetype('PPTelegraf-Regular.otf',size=40)
d.text((285,640), f"{total_plata}", font=fnt, fill=(0,0,0),align='center')


fnt = ImageFont.truetype('PPTelegraf-Regular.otf',size=40)
d.text((475,640), f"{total_bronce}", font=fnt, fill=(0,0,0),align='center')


from PIL import Image, ImageDraw, ImageOps

# Cargar la imagen de perfil
imagen_perfil = Image.open('avatar.png')  # Asume esta es la imagen de perfil a añadir

# Ajustar el tamaño de la imagen de perfil según sea necesario
tamaño_perfil = 316  # Este será el diámetro de tu círculo

# Aumentar el tamaño para el antialiasing simulado
factor_antialias = 4  # Factor por el cual aumentar el tamaño para mejorar el antialiasing
tamaño_antialias = tamaño_perfil * factor_antialias
imagen_perfil = imagen_perfil.resize((tamaño_antialias, tamaño_antialias), Image.Resampling.LANCZOS)

# Crear una máscara para la imagen de perfil con el tamaño aumentado
mascara = Image.new('L', (tamaño_antialias, tamaño_antialias), 0)
dibujar = ImageDraw.Draw(mascara)
dibujar.ellipse((0, 0, tamaño_antialias, tamaño_antialias), fill=255)

# Aplicar la máscara a la imagen de perfil
perfil_redondeado = Image.new('RGBA', (tamaño_antialias, tamaño_antialias), (0, 0, 0, 0))
perfil_redondeado.paste(imagen_perfil, (0, 0), mascara)

# Reducir el tamaño de nuevo al deseado para mejorar los bordes
perfil_redondeado = perfil_redondeado.resize((tamaño_perfil, tamaño_perfil), Image.Resampling.LANCZOS)

# Asumiendo que 'img' es tu imagen de fondo ya definida y cargada
# Posición donde quieras añadir la imagen de perfil en la imagen de fondo
posicion_x = 122  # Ajusta a tu preferencia
posicion_y = 180  # Ajusta a tu preferencia

# Pegar la imagen de perfil redondeada en la imagen de fondo
img.paste(perfil_redondeado, (posicion_x, posicion_y), perfil_redondeado)

# # Guardar la imagen modificada
img.save('Kaggle_card.png')


img.show()

In [ ]:
# !pip list --format=freeze > requirements.txt